# Time Series

## Persiapan Lingkungan



*   Download & Install Package

In [1]:
!pip install orchest
!pip install jupyterlab_code_formatter
!pip install pandas numpy scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 23.0 MB/s eta 0:00:00
  Created wheel for orchest: filename=orchest-0.3.11-py3-none-any.whl size=19345 sha256=5692dd5e812eebfb4296d4b5941c1cd91dda1c17aafd3f492fb2024b233df483
  Stored in directory: /root/.cache/pip/wheels/34/fb/fc/0cfd52dd71822472e9646034483e4add9139a6e62bd0e575fe
Successfully built orchest
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 9.0.0
    Uninstalling pyarrow-9.0.0:
      Successfully uninstalled pyarrow-9.0.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 5.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


*   Import Package

In [2]:
import orchest
import pandas as pd
import numpy as np
from numpy import array
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import os

*   [Dataset Passenger](https://raw.githubusercontent.com/dhamvi01/Univariate-Time-Series-using-LSTM/master/airline-passengers.csv)

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/dhamvi01/Univariate-Time-Series-using-LSTM/master/airline-passengers.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Month       144 non-null    object
 1   Passengers  144 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ KB


In [5]:
max_value, min_value = max(df["Passengers"]), min(df["Passengers"])
print(f"Nilai maksimal {max_value} dan nilai minimal {min_value} dari kolom 'Passengers'")

Nilai maksimal 622 dan nilai minimal 104 dari kolom 'Passengers'


## Preprocessing

Take ```Passengers``` column

In [6]:
passengers = array(df["Passengers"])



Bagi kolom menjadi _input_ dan _output_


*   ```idx_nilai_akhir = idx_iterasi + langkah``` ambil ```idx``` nilai terakhir
*   cek kondisi ```if idx_nilai_akhir > len(kolom)``` berhenti jika ```idx_nilai_akhir``` lebih besar dari panjang kolom
*   ```kolom-input kolom-output = kolom[i:idx_akhir], kolom[idx_akhir]``` ambil nilai _array_ sesuai dari indeks ```i``` sampai```idx_nilai_akhir```


In [7]:
def split_sequence(sequence, n_steps):
    """
    Split a univariate sequence into samples
        end_ix = i + n_steps # find the end of this pattern
        if end_ix > len(sequence)-1 # check if we are beyond the sequence
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix] # gather input and output parts of the pattern
    """
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

ambil kolom _input_ dengan nama ```matrix_data```

In [8]:
data, data_actual = split_sequence(passengers, 3)
matrix_data = pd.DataFrame((data), columns=["Xt-2","Xt-1", "Xt"])
y_actual = pd.DataFrame((data_actual), columns=["output"])

[Hitungan](https://docs.google.com/spreadsheets/d/1-Aeaz4VK1eppbsd0klJYrCulL_PP_CdThG12nqEeaNQ/edit?usp=sharing) manual nilai error terhadap kolom ```Xt``` dengan ```pred_y```

In [9]:
matrix_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Xt-2    141 non-null    int64
 1   Xt-1    141 non-null    int64
 2   Xt      141 non-null    int64
dtypes: int64(3)
memory usage: 3.4 KB


Bagi ```matrix_data``` menjadi data latih ```X``` dan data target ```y```

In [10]:
X = matrix_data.drop(columns="Xt")
y = matrix_data["Xt"]

Normalisasi dengan ```MinMaxScaler()``` data latih ```X``` dan data target ```y```

In [11]:
scaler = MinMaxScaler()

X_norm = scaler.fit_transform(X)
y_norm = scaler.fit_transform(y.values.reshape(-1, 1)) # reshape to 1 dimentional column

In [12]:
split_percentage = 0.2

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=split_percentage, random_state=0)

In [14]:
# use from KNeighborsRegressor because the data is numeric, for count root mean squared error
neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)

In [15]:
pred_y = neigh.predict(X_test)

In [16]:
mse = mean_squared_error(y_test, pred_y)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.006128763185080589


In [17]:
# use squeze to make 1 dimentional array 
# https://numpy.org/doc/stable/reference/generated/numpy.squeeze.html

pred_y_series = pd.Series(pred_y.squeeze())
y_test_series = pd.Series(y_test.squeeze())
passengers_df_norm = pd.concat((pred_y_series, y_test_series), axis=1)
passengers_df_norm.columns = ["pred_y", "y_test"]

In [18]:
passengers_df_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pred_y  29 non-null     float64
 1   y_test  29 non-null     float64
dtypes: float64(2)
memory usage: 592.0 bytes


In [19]:
passengers_df_norm.to_csv("/content/drive/MyDrive/prosaindata/source/tasks/output/passengers-df-norm.csv")